# PA

https://dati-noipa.mef.gov.it/cl/web/open-data/dataset

# Init

In [ ]:
%run Init.ipynb


Check Versions

Selected Jupyter core packages...
IPython          : 8.6.0
ipykernel        : 6.17.0
ipywidgets       : 7.6.5
jupyter_client   : 7.4.4
jupyter_core     : 4.11.2
jupyter_server   : 1.23.0
jupyterlab       : 3.5.0
nbclient         : 0.7.0
nbconvert        : 7.2.3
nbformat         : 5.7.0
notebook         : 6.5.2
qtconsole        : 5.4.0
traitlets        : 5.5.0

python           : 3.9.13    
pandas           : 1.5.1     
numpy            : 1.22.4    
sklearn          : 1.1.3     


Set Path

Name: Analysis
Path: /home/bifani/GitHub/Analysis

sys.path: 7
['/home/bifani/GitHub/Analysis/scripts',
 '/home/bifani/GitHub/Analysis/notebooks',
 '/home/bifani/miniconda3/envs/ama/lib/python39.zip',
 '/home/bifani/miniconda3/envs/ama/lib/python3.9',
 '/home/bifani/miniconda3/envs/ama/lib/python3.9/lib-dynload',
 '',
 '/home/bifani/miniconda3/envs/ama/lib/python3.9/site-packages']

Adding Path to sys.path: 8
['/home/bifani/GitHub/Analysis/scripts',
 '/home/bifani/GitHub/Analysis/not

<IPython.core.display.Javascript object>

In [ ]:
!pip install sparql-dataframe

  Using cached sparql_dataframe-0.4-py3-none-any.whl (3.5 kB)
  Using cached SPARQLWrapper-2.0.0-py3-none-any.whl (28 kB)
  Using cached rdflib-6.2.0-py3-none-any.whl (500 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)


<IPython.core.display.Javascript object>

In [ ]:
from src.datafactory import DataFactory

ModuleNotFoundError: No module named 'pandasql'

<IPython.core.display.Javascript object>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import sparql_dataframe

In [ ]:
dir_data = "../data/Intellera/"
dir_save = "../../../Shared/Data"

date = "2022-09-01"

# Functions

In [ ]:
def get_dates(start="20180701", end="20221001"):
    dateRange = pd.date_range(start=start, end=end, freq="M")
    dates = list(dateRange.strftime("%Y%m"))
    return dates

In [ ]:
def sparql_query(q):
    endpoint = "https://sparql-noipa.mef.gov.it/sparql/"

    df = sparql_dataframe.get(endpoint, q)
    if df.shape[0] == 10000:
        print("WARNING - number of rows", df.shape[0])

    if "numero" in df.columns:
        if df["numero"].isnull().sum().sum():
            print("null", df["numero"].isnull().sum().sum())
        print("numero:", df["numero"].sum())
    return df

In [ ]:
def sparql_query_time(q, start="20180701", end="20221001", sleep=3):
    import time

    df = pd.DataFrame()
    for d in get_dates(start=start, end=end):
        print("data:", d)
        qq = q.replace("TIME", d)
        try:
            time.sleep(sleep)
            df_ = sparql_query(qq)
            if df.empty:
                df = df_
            else:
                df = pd.concat([df, df_], axis=0, ignore_index=True)
        except Exception as e:
            print(e)

    return df

In [ ]:
def set_time(df, col="ti"):
    df[col] = pd.to_datetime(df[col], format="%Y%m")
    return df

In [ ]:
def set_age(df, col_min="eta_min", col_max="eta_max", group=False):
    for c in [col_min, col_max]:
        df[c].fillna(100, inplace=True)
        df[c] = df[c].astype("int32")
        df[c] = df[c].astype("string")

    df["eta"] = df[col_min] + "-" + df[col_max]
    df["eta"] = df["eta"].astype("object")
    df = df.drop(columns=[col_min, col_max])

    if group:
        df["eta"] = df["eta"].str.replace("25-34", "25-44")
        df["eta"] = df["eta"].str.replace("35-44", "25-44")
        df["eta"] = df["eta"].str.replace("45-54", "45-64")
        df["eta"] = df["eta"].str.replace("55-64", "45-64")

    return df

In [ ]:
def set_geo(df, col="provincia"):
    #     https://www.istat.it/it/archivio/6789
    #     dfG = pd.read_excel(
    #         dir_data + "Codici-statistici-e-denominazioni-al-12_08_2022.xls"
    #     )
    dfG = pd.read_csv(
        dir_data + "Codici-statistici-e-denominazioni-al-12_08_2022.csv",
        sep=";",
        encoding="latin-1",
    )
    cols = [
        "Codice Provincia (Storico)(1)",
        # "Codice Regione",
        "Ripartizione geografica",
    ]
    dfG = dfG[cols]
    dfG = dfG.rename(
        columns={
            "Codice Provincia (Storico)(1)": "provincia",
            "Ripartizione geografica": "ripartizione",
        }
    )
    dfG[col] = dfG[col].astype("int32")
    dfG = dfG.drop_duplicates()

    if df.dtypes[col] == "O":
        df[col] = df[col].str.replace(
            "https://w3id.org/italia/controlled-vocabulary/territorial-classifications/provinces/",
            "",
        )
    df[col] = df[col].astype("int32")
    df = df.merge(dfG, on=col)

    return df

In [ ]:
def set_pop(df):
    #     http://dati.istat.it/Index.aspx?DataSetCode=DCIS_POPRES1&Lang=it
    dfP = pd.read_csv(
        dir_data + "DCIS_POPRES1_03112022120514166.csv",
    )

    dfP = dfP[
        (dfP["ETA1"] == "TOTAL")
        #     & (dfP["Sesso"] == "totale")
        & ((dfP["Sesso"] == "maschi") | (dfP["Sesso"] == "femmine"))
        & (dfP["ITTER107"].str.len() == 3)
    ]
    cols = [
        "Territorio",
        "Sesso",
        "Value",
    ]
    dfP = dfP[cols]
    dfP = dfP.rename(
        columns={
            "Territorio": "ripartizione",
            "Sesso": "sesso",
            "Value": "popolazione",
        }
    )
    dfP["sesso"] = dfP["sesso"].str.replace("maschi", "M")
    dfP["sesso"] = dfP["sesso"].str.replace("femmine", "F")
    dfP = dfP.reset_index(drop=True)

    df = df.merge(dfP, on=["ripartizione", "sesso"])
    df["numero"] = df["numero"] / df["popolazione"]
    df = df.drop(columns="popolazione")

    return df

In [ ]:
def set_strings(df, column):
    df[column] = df[column].str.replace(",", "")
    if column != "qualifica":
        df[column] = df[column].str.replace("-", "")
    df[column] = df[column].str.replace("  ", " ")
    df[column] = df[column].str.upper()
    df[column] = df[column].str.lstrip()
    df[column] = df[column].str.rstrip()
    return df

In [ ]:
def set_comparto(df):
    dfCQ = pd.read_excel(
        dir_data + "comparto_qualifiche_con tag.xlsx",
        sheet_name="Tabella comparto finale",
    )
    dfCQ = dfCQ.rename(columns={"qualifica_contrattuale": "qualifica"})
    dfCQ = dfCQ.drop(columns=["NOTE"])
    dfCQ = set_strings(dfCQ, "comparto")
    dfCQ = set_strings(dfCQ, "qualifica")

    df = set_strings(df, "comparto")
    df = set_strings(df, "qualifica")

    df_ = df.merge(dfCQ, on=["comparto", "qualifica"], how="inner")
    df_ = df_.drop(columns=["comparto", "qualifica"])
    df_ = df_.rename(columns={"comparto_new": "comparto", "qualifica_new": "qualifica"})
    df_ = df_.drop(columns=["qualifica"])
    return df_

In [ ]:
def set_livello(df, comb=False):
    column1 = "comparto"
    df = set_strings(df, column1)

    if comb:
        replace = {
            "AREA DIRIGENZIALE DEL CONSIGLIO DEI MINISTRI": "AREA DIRIGENZIALE FUNZIONI CENTRALI",
            "COMPARTO DEL CONSIGLIO DEI MINISTRI": "COMPARTO FUNZIONI CENTRALI",
            "PERSONALE DELLA MAGISTRATURA": "COMPARTO FUNZIONI CENTRALI",
        }
        for k, v in replace.items():
            df[column1] = df[column1].str.replace(k, v)

    column2 = "livello"
    df[column2] = df[column1]

    replace = {
        " DI ": "",
        " DELLA ": "",
        "AREA DIRIGENZIALE": "",
        "AREA DIRIRENGIALE": "",
        "DIRIGENZA": "",
        "COMPARTO": "",
        "PERSONALE": "",
        "MEDICINA SPECIALISTICA": "",
        "MEDICINA GENERALE": "",
    }
    for k, v in replace.items():
        df[column1] = df[column1].str.replace(k, v)

    replace = {
        "DIFESA SICUREZZA E SOCCORSO PUBBLICO": "",
        "ISTRUZIONE E RICERCA": "",
        "FUNZIONI CENTRALI": "",
        "SANITA'": "",
        "FUNZIONI LOCALI": "",
        "PRESIDENZA DEL CONSIGLIO DEI MINISTRI": "",
        "MAGISTRATURA": "",
        "ADDETTO SISTEMAZIONE IDRAULICA FORESTALE E AGRARIA": "",
        "ENTI PUBBLICI ECONOMICI": "",
        "REGIONI AUTONOME": "",
        "COLLABORAZIONE": "",
        "PERSONALE DI": "PERSONALE",
        "PERSONALE DELLA": "PERSONALE",
        "COMPARTO": "PERSONALE",
        "MEDICINA SPECIALISTICA": "PERSONALE",
        "MEDICINA GENERALE": "PERSONALE",
        "AUTORITA' INDIPENDENTI": "PERSONALE",
        "AREA DIRIGENZIALE": "DIRIGENZA",
        "AREA DIRIRENGIALE": "DIRIGENZA",
    }
    for k, v in replace.items():
        df[column2] = df[column2].str.replace(k, v)

    df = set_strings(df, column1)
    df = set_strings(df, column2)
    #     df = df.sort_values(column)
    return df

In [ ]:
def set_sex(df, col="sesso", col_ext="ti", drop=None, force=False, frac=False):
    cols = [col]
    if isinstance(col_ext, list):
        cols.extend(col_ext)
    else:
        cols.append(col_ext)

    df1 = df.groupby(cols).sum()
    df1 = df1.reset_index()
    df1 = df1.set_index(col_ext)

    dfM = df1[df1[col] == "M"]
    dfM = dfM.drop(columns=col)
    dfM = dfM.rename(columns={"numero": "M"})

    dfF = df1[df1[col] == "F"]
    dfF = dfF.drop(columns=col)
    dfF = dfF.rename(columns={"numero": "F"})

    if force:
        for c in ["eta", "ripartizione", "provincia"]:
            if c not in col_ext:
                if c in dfM.columns:
                    dfM = dfM.drop(columns=[c])
                if c in dfF.columns:
                    dfF = dfF.drop(columns=[c])

    df2 = dfM.join(dfF, how="inner")
    df2["F/M"] = df2["F"] / df2["M"]
    #     df2["F+M"] = df2["F"] + df2["M"]

    df2 = df2.reset_index()
    if frac:
        df2["T"] = df2["M"] + df2["F"]
        df2["M"] = df2["M"] / df2["T"]
        df2["F"] = df2["F"] / df2["T"]
        df2 = df2.drop(columns=["T"])
    if drop:
        df2 = df2.drop(columns=["M", "F"])
    df2 = df2.set_index(col_ext).sort_index()

    return df2

In [ ]:
def set_ral(
    df,
    col_imp="totImponibile",
    col_rit="totRitenute",
    col_ced="totCedolini",
    cols=["sesso", "ti", "eta"],
    tot=False,
    ced=False,
):
    df[col_ced] = df[col_ced].astype("int32")
    df = df.groupby(cols).sum()

    df["perc"] = (df[col_imp] - df[col_rit]) / df[col_imp]
    df["imponibile"] = df[col_imp] / df[col_ced]
    df["ritenute"] = df[col_rit] / df[col_ced]
    df["numero"] = df["imponibile"]
    if tot:
        df["numero"] = df[col_imp]
    if ced:
        df["numero"] = df[col_ced]
    df = df.drop(columns=[col_imp, col_rit, col_ced])
    df = df.drop(columns=["perc", "imponibile", "ritenute"])

    return df

# Datasets

## Amministrati

In [ ]:
q = """
PREFIX agerange:<https://sparql-noipa.mef.gov.it/resources/AgeRange/>
PREFIX geonames:<http://www.geonames.org/ontology#>
SELECT ?provincia ?eta_min ?eta_max ?sesso ?ti (sum(?numero) as ?numero )
WHERE {
?entry a npont:EntryAmministrati .
?time npont:temporalID ?ti. FILTER( ?ti=TIME )
?entry npont:hasTime ?time .
?time npont:temporalID ?ti.
?entry npont:howMany ?numero .
?entry npont:hasPlace ?p .
?p rdfs:label ?comune .
?p geonames:locatedIn ?provincia .
?entry npont:genderType ?sesso .
?entry npont:hasAgeRange ?ar .
OPTIONAL{ ?ar npont:minAge ?eta_min . }
OPTIONAL{ ?ar npont:maxAge ?eta_max . }
}
ORDER BY DESC(?amministrati)
"""

dfA = sparql_query_time(q, "20180701", "20221001", sleep=5)
# dfA = sparql_query_time(q, "20220101", "20220201", sleep=5)
dfA

In [ ]:
dfA.to_csv(dir_data + "Amministrati.csv")

## ContrattiGestiti

In [ ]:
q = """
PREFIX provincia:<http://spcdata.digitpa.gov.it/Provincia/>
PREFIX geonames:<http://www.geonames.org/ontology#>
SELECT ?comparto ?qualifica ?eta_min ?eta_max ?sesso ?ti SUM(?numero) as ?numero
WHERE {
 ?entry npont:hasTime ?time .
 ?time npont:temporalID ?ti. FILTER( ?ti=TIME )
 ?entry a npont:EntryContrattiGestiti .
 ?entry npont:hasPlace ?uri_locazione .
 ?entry npont:inEnte ?ente .
 ?ente rdfs:label ?ente_ .
 ?entry npont:conCodiceComparto ?c .
 ?c rdfs:label ?comparto .
 ?entry npont:conQualificaContrattuale ?c2 .
 ?c2 rdfs:label ?qualifica .
 ?entry npont:howMany ?numero .
 ?entry npont:hasAgeRange ?ar .
 ?entry npont:genderType ?sesso .
 OPTIONAL{ ?ar npont:minAge ?eta_min . }
 OPTIONAL{ ?ar npont:maxAge ?eta_max . }
} 
ORDER BY DESC(?amministrati)
"""

dfCG = sparql_query_time(q, "20180701", "20221001", sleep=5)
# dfCG = sparql_query_time(q, "20220101", "20220201", sleep=5)
dfCG

In [ ]:
dfCG.to_csv(dir_data + "ContrattiGestiti.csv")

In [ ]:
q = """
PREFIX provincia:<http://spcdata.digitpa.gov.it/Provincia/>
PREFIX geonames:<http://www.geonames.org/ontology#>
SELECT ?provincia ?eta_min ?eta_max ?sesso ?ti SUM(?numero) as ?numero
WHERE {
 ?entry npont:hasTime ?time .
 ?time npont:temporalID ?ti. FILTER( ?ti=TIME )
 ?entry a npont:EntryContrattiGestiti .
 ?entry npont:hasPlace ?p .
 ?p rdfs:label ?comune .
 ?p geonames:locatedIn ?provincia .
 ?entry npont:inEnte ?ente .
 ?ente rdfs:label ?ente_ .
 ?entry npont:conCodiceComparto ?c .
 ?c rdfs:label ?comparto .
 ?entry npont:conQualificaContrattuale ?c2 .
 ?c2 rdfs:label ?qualifica .
 ?entry npont:howMany ?numero .
 ?entry npont:hasAgeRange ?ar .
 ?entry npont:genderType ?sesso .
 OPTIONAL{ ?ar npont:minAge ?eta_min . }
 OPTIONAL{ ?ar npont:maxAge ?eta_max . }
} 
ORDER BY DESC(?amministrati)
"""

dfCG_2 = sparql_query_time(q, "20180701", "20221001", sleep=5)
# dfCG_2 = sparql_query_time(q, "20220101", "20220201", sleep=5)
dfCG_2

In [ ]:
dfCG_2.to_csv(dir_data + "ContrattiGestiti2.csv")

## MotivoAssunzione

In [ ]:
q = """
PREFIX agerange:<https://sparql-noipa.mef.gov.it/resources/AgeRange/>
PREFIX geonames:<http://www.geonames.org/ontology#>
SELECT ?motivo_ ?eta_min ?eta_max ?sesso ?ti SUM(?numero) as ?numero
WHERE{
 ?entry npont:hasTime ?time .
 ?time npont:temporalID ?ti. FILTER( ?ti=TIME )
 ?entry a npont:EntryMotivoAssunzione .
 ?entry npont:howMany ?numero .
 ?entry npont:hasPlace ?uri_provincia .
 ?uri_provincia rdfs:label ?provincia .
 ?entry npont:genderType ?sesso .
 ?entry npont:hasAgeRange ?ar .
 OPTIONAL{ ?ar npont:minAge ?eta_min . }
 OPTIONAL{ ?ar npont:maxAge ?eta_max . }
 ?entry npont:motivoAssunzione ?motivo .
 ?motivo rdfs:label ?motivo_ . 
}
ORDER BY DESC(?amministrati)
"""

dfMA = sparql_query_time(q, "20180701", "20221001", sleep=5)
# dfMA = sparql_query_time(q, "20220101", "20220201", sleep=5)
dfMA

In [ ]:
dfMA.to_csv(dir_data + "MotivoAssunzione.csv")

## MotivoCessazione

In [ ]:
q = """
PREFIX agerange:<https://sparql-noipa.mef.gov.it/resources/AgeRange/>
PREFIX geonames:<http://www.geonames.org/ontology#>
SELECT ?motivo_ ?eta_min ?eta_max ?sesso ?ti SUM(?numero) as ?numero
WHERE{
 ?entry npont:hasTime ?time .
 ?time npont:temporalID ?ti. FILTER( ?ti=TIME )
 ?entry a npont:EntryMotivoCessazione .
 ?entry npont:howMany ?numero .
 ?entry npont:hasPlace ?uri_provincia .
 ?uri_provincia rdfs:label ?provincia .
 ?entry npont:genderType ?sesso .
 ?entry npont:hasAgeRange ?ar .
 OPTIONAL{ ?ar npont:minAge ?eta_min . }
 OPTIONAL{ ?ar npont:maxAge ?eta_max . }
 ?entry npont:motivoCessazione ?motivo .
 ?motivo rdfs:label ?motivo_ . 
}
ORDER BY DESC(?amministrati)
"""

dfMC = sparql_query_time(q, "20180701", "20221001", sleep=5)
# dfMC = sparql_query_time(q, "20220101", "20220201", sleep=5)
dfMC

In [ ]:
dfMC.to_csv(dir_data + "MotivoCessazione.csv")

## RitenuteFiscali

In [ ]:
q = """
#Imponibile e ritenute fiscali per la mensilità corrente degli amministrati afferenti al Ministero dell'Economia e Finanze
PREFIX ente:<https://sparql-noipa.mef.gov.it/resources/Ente/>
SELECT ?eta_min ?eta_max ?sesso ?ti SUM(?imponibile) as ?totImponibile SUM(?ritenute) as ?totRitenute SUM(?cedolini) as ?totCedolini
WHERE{
 ?entry npont:hasTime ?time .
 ?time npont:temporalID ?ti. FILTER( ?ti=TIME ) 
 ?entry a npont:EntryCedolinoRitenuteFiscali .
 ?entry npont:hasPlace ?uri_comune .
 ?uri_comune rdfs:label ?comune .
 ?entry npont:inEnte ?ente .
 ?ente rdfs:label ?ente_ .
# ?entry npont:conCodiceComparto ?c .
# ?c rdfs:label ?comparto .
 ?entry npont:imponibileIrpef ?imponibile .
 ?entry npont:ritenuteIrpef ?ritenute .
 ?entry npont:howManyCedolini ?cedolini .
 ?entry npont:hasAgeRange ?ar .
 ?entry npont:genderType ?sesso .
 OPTIONAL{ ?ar npont:minAge ?eta_min . }
 OPTIONAL{ ?ar npont:maxAge ?eta_max . }
}
ORDER BY DESC(?totRitenute)
"""

dfCF = sparql_query_time(q, "20180701", "20221001", sleep=5)
# dfCF = sparql_query_time(q, "20220101", "20220201", sleep=5)
dfCF

In [ ]:
dfCF.to_csv(dir_data + "RitenuteFiscali.csv")

# Amministrati

In [ ]:
# dfA1 = dfA.copy(deep=True)
dfA1 = pd.read_csv(dir_data + "Amministrati.csv", index_col="Unnamed: 0")
dfA1

In [ ]:
dfA1 = set_time(dfA1)
dfA1 = set_age(dfA1)
dfA1 = set_geo(dfA1)
dfA1

In [ ]:
dfA2 = set_sex(dfA1, col_ext=["ti"], force=True)
dfA2

## Totali

In [ ]:
dfA2.loc[date].to_frame().T[["F", "M"]].reset_index()  # .plot.barh(rot=0)

In [ ]:
# plt.figure()
ax = dfA2.loc[date].to_frame().T.reset_index()[["F", "M"]].plot.barh(xlim=(0, 1.4e6))

# plt.bar_label(ax, labels=[f"{x}" for x in ax.datavalues])

for container in ax.containers:
    #     c = f"{container.datavalues[0]:,}".replace(".0", "")
    ax.bar_label(container, fmt="  %d")

plt.title("Amministrati in PA")
plt.xlabel("Numero amministrati")
plt.ylabel("")
plt.tight_layout()

fig = ax.get_figure()
fig.savefig(dir_save + "totale.jpg")

In [ ]:
# plt.figure()
ax = dfA2.loc[date].to_frame().T.reset_index()[["F", "M"]].T.plot.pie(subplots=True, autopct="%1.1f%%")

plt.title("Amministrati in PA")
# plt.xlabel("Numero amministrati")
plt.ylabel("")
plt.tight_layout()

fig = ax.get_figure()
fig.savefig(dir_save + "totale_pie.jpg")

In [ ]:
dfA2[["F", "M"]].plot(ylim=(0.5e6, 2.5e6))
plt.tight_layout()

In [ ]:
dfA2[["F/M"]].plot(ylim=(1, 2))
plt.tight_layout()

## Età

In [ ]:
dfA3 = set_sex(dfA1, col_ext=["ti", "eta"], drop=True, force=True)
dfA3

In [ ]:
dfA3.unstack().plot(ylim=(0, 2.5))
plt.legend(ncol=3)
plt.tight_layout()

In [ ]:
dfA5 = set_sex(dfA1, col_ext=["ti", "eta"], force=True)
dfA5

In [ ]:
ax = (
    dfA5.loc[date][["F", "M"]]
    .reindex(index=["65-100", "55-64", "45-54", "35-44", "25-34", "18-24"])
    .plot.barh(xlim=(0, 5e5))
)

for container in ax.containers:
    ax.bar_label(container, fmt="  %d")

plt.title("Amministrati in PA")
plt.xlabel("Numero amministrati")
plt.ylabel("Fascia di età")
plt.tight_layout()

fig = ax.get_figure()
fig.savefig(dir_save + "eta.jpg")

In [ ]:
for g in ["F", "M"]:
    plt.figure()
    fig = dfA5.loc[date][g].plot.pie(figsize=(10, 10), subplots=True, autopct="%1.1f%%")

    plt.title("Amministrati in PA (" + g + ")")
    #     plt.xlabel("Numero amministrati")
    plt.ylabel("")
    plt.tight_layout()

    #     fig = ax.get_figure()
#     fig.savefig(dir_save + "eta_" + g + ".jpg")

## Regione

In [ ]:
dfA6 = set_sex(dfA1, col_ext=["ti", "ripartizione"], force=True)
dfA6

In [ ]:
dfA6["F/M"].unstack(1).plot(figsize=(8, 8), ylim=(0, 3))
plt.tight_layout()

In [ ]:
ax = (
    dfA6.loc[date][["F", "M"]]
    .reindex(index=["Nord-ovest", "Nord-est", "Centro", "Sud", "Isole"])
    .iloc[::-1]
    .plot.barh(figsize=(8, 8), xlim=(0, 4.5e5))
)

for container in ax.containers:
    ax.bar_label(container, fmt="  %d")

plt.title("Amministrati in PA")
plt.xlabel("Numero amministrati")
plt.ylabel("Gruppo di regioni")
plt.tight_layout()

fig = ax.get_figure()
fig.savefig(dir_save + "regione.jpg")

In [ ]:
for g in ["F", "M"]:
    plt.figure()
    #     fig = dfA5.loc[date][g].plot.pie(
    #         figsize=(10, 10), subplots=True, autopct="%1.1f%%"
    #     )
    fig = dfA6.loc[date][g].plot.pie(figsize=(10, 10), subplots=True, autopct="%1.1f%%")

    plt.title("Amministrati in PA (" + g + ")")
    #     plt.xlabel("Numero amministrati")
    plt.ylabel("")
    plt.tight_layout()

    #     fig = ax.get_figure()
#     fig.savefig(dir_save + "eta_" + g + ".jpg")

### ISTAT

In [ ]:
dfA10 = pd.read_csv(dir_data + "Amministrati.csv", index_col="Unnamed: 0")

In [ ]:
dfA10 = set_time(dfA10)
dfA10 = set_age(dfA10)
dfA10 = set_geo(dfA10)
dfA10 = set_pop(dfA10)
dfA10

In [ ]:
dfA11 = set_sex(dfA10, col_ext=["ti", "ripartizione"], force=True)
dfA11

In [ ]:
dfA12 = dfA11["F/M"].unstack(1)
dfA12.plot(figsize=(8, 8), ylim=(0, 3))
plt.tight_layout()

# ContrattiGestiti

In [ ]:
# dfCG1 = dfCG.copy(deep=True)
dfCG1 = pd.read_csv(dir_data + "ContrattiGestiti.csv", index_col="Unnamed: 0")
dfCG1

In [ ]:
C = sorted(dfCG1["comparto"].unique())
print(len(C))
C

In [ ]:
Q = sorted(dfCG1["qualifica"].unique())
print(len(Q))
Q

In [ ]:
dfCG1 = set_time(dfCG1)
dfCG1 = set_age(dfCG1)
dfCG1

## Comparto-Qualifica

In [ ]:
dfCG1 = set_strings(dfCG1, "comparto")
dfCG1 = set_strings(dfCG1, "qualifica")

In [ ]:
dfCG100 = set_comparto(dfCG1)
dfCG100

In [ ]:
def set_livello(df, comb=False):
    column1 = "comparto"
    df = set_strings(df, column1)

    if comb:
        replace = {
            "AREA DIRIGENZIALE DEL CONSIGLIO DEI MINISTRI": "AREA DIRIGENZIALE FUNZIONI CENTRALI",
            "COMPARTO DEL CONSIGLIO DEI MINISTRI": "COMPARTO FUNZIONI CENTRALI",
            "PERSONALE DELLA MAGISTRATURA": "COMPARTO FUNZIONI CENTRALI",
        }
        for k, v in replace.items():
            df[column1] = df[column1].str.replace(k, v)

    column2 = "livello"
    df[column2] = df[column1]

    replace = {
        " DI ": "",
        " DELLA ": "",
        "AREA DIRIGENZIALE": "",
        "AREA DIRIRENGIALE": "",
        "DIRIGENZA": "",
        "COMPARTO": "",
        "PERSONALE": "",
        "MEDICINA SPECIALISTICA": "",
        "MEDICINA GENERALE": "",
    }
    for k, v in replace.items():
        df[column1] = df[column1].str.replace(k, v)

    replace = {
        "DIFESA SICUREZZA E SOCCORSO PUBBLICO": "",
        "ISTRUZIONE E RICERCA": "",
        "FUNZIONI CENTRALI": "",
        "SANITA'": "",
        "FUNZIONI LOCALI": "",
        "PRESIDENZA DEL CONSIGLIO DEI MINISTRI": "",
        "MAGISTRATURA": "",
        "ADDETTO SISTEMAZIONE IDRAULICA FORESTALE E AGRARIA": "",
        "ENTI PUBBLICI ECONOMICI": "",
        "REGIONI AUTONOME": "",
        "COLLABORAZIONE": "",
        "PERSONALE DI": "PERSONALE",
        "PERSONALE DELLA": "PERSONALE",
        "COMPARTO": "PERSONALE",
        "MEDICINA SPECIALISTICA": "PERSONALE",
        "MEDICINA GENERALE": "PERSONALE",
        "AUTORITA' INDIPENDENTI": "PERSONALE",
        "AREA DIRIGENZIALE": "DIRIGENZA",
        "AREA DIRIRENGIALE": "DIRIGENZA",
    }
    for k, v in replace.items():
        df[column2] = df[column2].str.replace(k, v)

    df = set_strings(df, column1)
    df = set_strings(df, column2)
    #     df = df.sort_values(column)
    return df

In [ ]:
dfCG101 = set_livello(dfCG100, comb=True)
dfCG101

In [ ]:
dfCG101

## Time

In [ ]:
dfCG1 = dfCG1.groupby(["comparto", "ti", "sesso"]).sum().reset_index()
dfCG1

In [ ]:
dfCG2 = set_sex(dfCG1, col_ext=["comparto", "ti"], drop=True)
dfCG2

In [ ]:
dfCG3 = dfCG2.unstack(0)

In [ ]:
for c in CC:
    print(c)
    dfCG3.iloc[:, dfCG3.columns.get_level_values(1).str.startswith(c)].plot(ylim=(0, 5))
    plt.tight_layout()

In [ ]:
for c in CCC:
    print(c)
    dfCG3.iloc[:, dfCG3.columns.get_level_values(1).str.contains(c)].plot(ylim=(0, 5))
    plt.tight_layout()

In [ ]:
dfCG3.plot(figsize=(8, 8), ylim=(0, 6), colormap="tab20")
plt.legend(ncol=2, fontsize="xx-small")
plt.tight_layout()

## Time 2

In [ ]:
dfCG1_ = dfCG101.groupby(["comparto", "ti", "sesso"]).sum().reset_index()
dfCG1_

In [ ]:
dfCG102 = set_sex(dfCG1_, col_ext=["comparto", "ti"], drop=False, frac=False)
dfCG102

In [ ]:
dfCG102.reset_index()["comparto"].value_counts()

In [ ]:
c = "COMPARTO FUNZIONI CENTRALI"
idx = pd.IndexSlice
ax = dfCG102.unstack(0).loc[:, idx[["F", "M"], c]].plot()
ax.legend(["F", "M"])

plt.title("Amministrati in PA - " + c)
plt.xlabel("")
plt.ylabel("Numero amministrati")
plt.tight_layout()

In [ ]:
c = "COMPARTO ISTRUZIONE E RICERCA"
idx = pd.IndexSlice
ax = dfCG102.unstack(0).loc[:, idx[["F", "M"], c]].plot()
ax.legend(["F", "M"])

plt.title("Amministrati in PA - " + c)
plt.xlabel("")
plt.ylabel("Numero amministrati")
plt.tight_layout()

## Bars

In [ ]:
dfCG5 = set_sex(dfCG1, col_ext=["comparto", "ti"], drop=False)
dfCG5 = dfCG5.drop(columns="F/M")
dfCG5 = dfCG5.reset_index()
dfCG5 = dfCG5[["comparto", "ti", "F", "M"]]
dfCG5 = dfCG5[dfCG5["ti"] == date]
dfCG5 = dfCG5.drop(columns="ti")
dfCG5

In [ ]:
dfCG5 = set_livello(dfCG5)
dfCG5["comparto"]

In [ ]:
dfCG5 = dfCG5.set_index("comparto")
# dfCG5
dfCG5.plot.bar(figsize=(10, 10), rot=90, logy=True)
plt.tight_layout()

In [ ]:
dfCG6 = pd.DataFrame(columns=dfCG5.columns)
for c in [
    "DIRIGENZA",
    "PERSONALE",
]:
    dfCG6.loc[c] = dfCG5.loc[dfCG5.index.str.contains(c), :].sum()

dfCG6

In [ ]:
ax = dfCG6.reindex(
    index=[
        "PERSONALE",
        "DIRIGENZA",
    ]
).plot.barh(xlim=(0, 2.5e6), logx=False)

for container in ax.containers:
    ax.bar_label(container, fmt="  %d")

plt.title("Amministrati in PA")
plt.xlabel("Numero amministrati")
plt.ylabel("")
plt.tight_layout()

fig = ax.get_figure()
fig.savefig(dir_save + "comparto.jpg")

In [ ]:
for c in ["FUNZIONI C", "FUNZIONI L", "ISTRUZIONE", "PRESIDENZA", "SANITA", "ENTI P"]:
    print(c)
    dfCG5.loc[dfCG5.index.str.contains(c), :].plot.bar(figsize=(8, 6), rot=45, logy=True)
    plt.tight_layout()

In [ ]:
c = "FUNZIONI CENTRALI"
ax = (
    dfCG5.loc[dfCG5.index.str.contains(c), :]
    .reindex(
        index=[
            "PERSONALE - " + c,
            "DIRIGENZA - " + c,
        ]
    )
    .plot.barh(xlim=(0, 1.1e5), logy=False)
)
for container in ax.containers:
    ax.bar_label(container, fmt="  %d")

plt.title("Amministrati in PA - " + c)
plt.xlabel("Numero amministrati")
plt.ylabel("")
labels = [t.get_text().replace(" - " + c, "") for t in ax.get_yticklabels()]
ax.set_yticklabels(labels)
plt.tight_layout()

fig = ax.get_figure()
fig.savefig(dir_save + "comparto_" + c + ".jpg")

In [ ]:
c = "FUNZIONI LOCALI"
ax = (
    dfCG5.loc[dfCG5.index.str.contains(c), :]
    .reindex(
        index=[
            "PERSONALE - " + c,
            "DIRIGENZA - " + c,
        ]
    )
    .plot.barh(xlim=(0, 3.5e3), logy=False)
)
for container in ax.containers:
    ax.bar_label(container, fmt="  %d")

plt.title("Amministrati in PA - " + c)
plt.xlabel("Numero amministrati")
plt.ylabel("")
labels = [t.get_text().replace(" - " + c, "") for t in ax.get_yticklabels()]
ax.set_yticklabels(labels)
plt.tight_layout()

fig = ax.get_figure()
fig.savefig(dir_save + "comparto_" + c + ".jpg")

In [ ]:
c = "ISTRUZIONE E RICERCA"
ax = (
    dfCG5.loc[dfCG5.index.str.contains(c), :]
    .reindex(
        index=[
            "PERSONALE - " + c,
            "DIRIGENZA - " + c,
        ]
    )
    .plot.barh(xlim=(0, 2.0e6), logy=False)
)
for container in ax.containers:
    ax.bar_label(container, fmt="  %d")

plt.title("Amministrati in PA - ISTRUZIONE E RICERCA")
plt.xlabel("Numero amministrati")
plt.ylabel("")
labels = [t.get_text().replace(" - " + c, "") for t in ax.get_yticklabels()]
ax.set_yticklabels(labels)
plt.tight_layout()

fig = ax.get_figure()
fig.savefig(dir_save + "comparto_" + c + ".jpg")

In [ ]:
c = "PRESIDENZA DEL CONSIGLIO DEI MINISTRI"
ax = (
    dfCG5.loc[dfCG5.index.str.contains(c), :]
    .reindex(
        index=[
            "PERSONALE - " + c,
            "DIRIGENZA - " + c,
        ]
    )
    .plot.barh(figsize=(7, 5), xlim=(0, 1.2e3), logy=False)
)
for container in ax.containers:
    ax.bar_label(container, fmt="  %d")

plt.title("Amministrati in PA - " + c)
plt.xlabel("Numero amministrati")
plt.ylabel("")
labels = [t.get_text().replace(" - " + c, "") for t in ax.get_yticklabels()]
ax.set_yticklabels(labels)
plt.tight_layout()

fig = ax.get_figure()
fig.savefig(dir_save + "comparto_" + c + ".jpg")

In [ ]:
c = "SANITA'"
ax = (
    dfCG5.loc[dfCG5.index.str.contains(c), :]
    .reindex(
        index=[
            "PERSONALE - " + c,
            "DIRIGENZA - " + c,
        ]
    )
    .plot.barh(xlim=(0, 2.5e4), logy=False)
)
for container in ax.containers:
    ax.bar_label(container, fmt="  %d")

plt.title("Amministrati in PA - " + c)
plt.xlabel("Numero amministrati")
plt.ylabel("")
labels = [t.get_text().replace(" - " + c, "") for t in ax.get_yticklabels()]
ax.set_yticklabels(labels)
plt.tight_layout()

fig = ax.get_figure()
fig.savefig(dir_save + "comparto_" + c + ".jpg")

In [ ]:
c = "ENTI PUBBLICI ECONOMICI"
ax = (
    dfCG5.loc[dfCG5.index.str.contains(c), :]
    .reindex(
        index=[
            "PERSONALE - " + c,
            "DIRIGENZA - " + c,
        ]
    )
    .plot.barh(xlim=(0, 0.7e3), logy=False)
)

for container in ax.containers:
    ax.bar_label(container, fmt="  %d")

plt.title("Amministrati in PA - " + c)
plt.xlabel("Numero amministrati")
plt.ylabel("")
labels = [t.get_text().replace(" - " + c, "") for t in ax.get_yticklabels()]
ax.set_yticklabels(labels)
plt.tight_layout()

fig = ax.get_figure()
fig.savefig(dir_save + "comparto_" + c + ".jpg")

In [ ]:
dfCG10 = pd.read_csv(dir_data + "ContrattiGestiti.csv", index_col="Unnamed: 0")
dfCG10

In [ ]:
dfCG10 = set_time(dfCG10)
dfCG10 = set_age(dfCG10)
dfCG10

In [ ]:
dfCG10 = dfCG10.groupby(["comparto", "ti", "sesso"]).sum().reset_index()
dfCG10 = dfCG10.drop(columns="comparto")
dfCG10

In [ ]:
dfCG11 = set_sex(dfCG10, col_ext=["ti"], drop=False)
dfCG11

In [ ]:
dfCG11[["F", "M"]].plot(ylim=(0.5e6, 2.5e6))
plt.tight_layout()

# MotivoAssunzione

In [ ]:
# dfMA1 = dfMA.copy(deep=True)
dfMA1 = pd.read_csv(dir_data + "MotivoAssunzione.csv", index_col="Unnamed: 0")
dfMA1

In [ ]:
MA = sorted(dfMA["motivo_"].unique())
print(len(MA))
MA

In [ ]:
dfMA1 = set_time(dfMA1)
dfMA1 = set_age(dfMA1)
dfMA1

In [ ]:
dfMA2 = set_sex(dfMA1, col_ext=["motivo_", "ti"], drop=True, force=True)
dfMA2

In [ ]:
dfMA3 = dfMA2.unstack(0)

## Time

In [ ]:
dfMA3.plot(figsize=(8, 8), ylim=(0, 4), colormap="tab20")
plt.legend(ncol=3, fontsize="x-small")
plt.tight_layout()

# MotivoCessazione

In [ ]:
# dfMC1 = dfMC.copy(deep=True)
dfMC1 = pd.read_csv(dir_data + "MotivoCessazione.csv", index_col="Unnamed: 0")
dfMC1

In [ ]:
MC = sorted(dfMC["motivo_"].unique())
print(len(MC))
MC

In [ ]:
dfMC1 = set_time(dfMC1)
dfMC1 = set_age(dfMC1)
dfMC1

In [ ]:
dfMC2 = set_sex(dfMC1, col_ext=["motivo_", "ti"], drop=False, force=True)
dfMC2

In [ ]:
dfMC3 = dfMC2.unstack(0)
dfMC3

## Time

In [ ]:
idx = pd.IndexSlice
dfMC3.loc[:, idx["F/M"]].plot(figsize=(8, 8), ylim=(0, 4), colormap="tab20")
plt.legend(ncol=2, fontsize="xx-small")
plt.tight_layout()

In [ ]:
cols = [
    #  'ALTRO MOTIVO',
    #  'CAMBIO COMPARTO/CONTRATTO',
    #  'COLLOCAMENTO ANTICIPATO (DLGS 503/92)',
    #  "DECADENZA DALL'IMPIEGO PER ACCETTAZIONE INCARICO A TEMPO DETERMINATO",
    #  'DECESSO',
    "DESTITUZIONE",
    #     "DIMISSIONI VOLONTARIE/RECESSO DEL DIPENDENTE",
    #  "DISPENSA DAL SERVIZIO PER INABILITA' ASSOLUTA PERMANENTE AD OGNI PROFICUO LAVORO",
    #  "DISPENSA DAL SERVIZIO PER INABILITA' PERMANENTE ALLE MANSIONI",
    #  "DISPENSA DAL SERVIZIO PER INVALIDITA' (ART. 2, COMMA 12, L. 335/95)",
    #     "FINE COMANDO",
    #     "FINE INCARICO",
    #  'FINE INCARICO DIRIGENZIALE (DLGS 165/01- ART.19,6)',
    "LICENZIAMENTO",
    #     "LIMITI DI ETA'",
    #     "LIMITI DI SERVIZIO",
    #  'MALATTIE DOVUTE A CAUSA DI SERVIZIO',
    "MOBILITA'",
    "PASSAGGIO A DIRIGENTE",
    #  'PASSAGGIO AD ALTRA AMMINISTRAZIONE',
    #  'PASSAGGIO ALTRA AMMINISTRAZIONE PER COMANDO',
    #  'PERDITA DELLA CITTADINANZA NEL RISPETTO DELLA NORMATIVA COMUNITARIA ',
    "RECESSO O MANCATO SUPERAMENTO PERIODO DI PROVA",
    #     "RIDUZIONE DOTAZIONE ORGANICA",
    #     "RISOLUZIONE CONSENSUALE PER I DIRIGENTI ",
    #  'SOSPESO - ASPETT. PER PERCORSO FIT - DL73'
]

In [ ]:
idx = pd.IndexSlice
for c in cols:
    dfMC3.loc[:, idx[["F/M"], c]].plot(ylim=(0, 2))
    plt.tight_layout()

In [ ]:
idx = pd.IndexSlice
for c in cols:
    dfMC3.loc[:, idx[["F", "M"], c]].plot()
    plt.tight_layout()

In [ ]:
idx = pd.IndexSlice
ax = dfMC3.loc[:, idx[["F", "M"], "PASSAGGIO A DIRIGENTE"]].plot()
ax.legend(["F", "M"])

for container in ax.containers:
    ax.bar_label(container, fmt="  %d")

plt.title("Amministrati in PA - PASSAGGIO A DIRIGENTE")
plt.xlabel("")
plt.ylabel("Numero amministrati passati a dirigente")
plt.tight_layout()

fig = ax.get_figure()
fig.savefig(dir_save + "PASSAGGIO A DIRIGENTE.jpg")

# RitenuteFiscali

In [ ]:
# dfCF1 = dfCF.copy(deep=True)
dfCF1 = pd.read_csv(dir_data + "RitenuteFiscali.csv", index_col="Unnamed: 0")
dfCF1

In [ ]:
dfCF[dfCF.columns[dfCF.columns.str.contains("tot")]].isnull().sum().sum()

In [ ]:
dfCF1 = set_time(dfCF1)
dfCF1 = set_age(dfCF1, group=True)
dfCF1

In [ ]:
dfCF2 = set_ral(dfCF1)
dfCF2

In [ ]:
dfCF3 = set_sex(dfCF2, col_ext=["ti", "eta"], drop=False)
dfCF3

In [ ]:
dfCF4 = dfCF3.unstack(1)
dfCF4

In [ ]:
idx = pd.IndexSlice
dfCF4.loc[:, idx["F/M"]].plot()
plt.tight_layout()

In [ ]:
idx = pd.IndexSlice
for c in ["18-24", "25-44", "45-64", "65-100"]:
    dfCF4.loc[:, idx[["F", "M"], c]].plot(logy=False)
    plt.tight_layout()

In [ ]:
idx = pd.IndexSlice
for c in ["18-24", "25-44", "45-64", "65-100"]:
    ax = dfCF4.loc[:, idx[["F", "M"], c]].plot(logy=False)
    ax.legend(["F", "M"])

    for container in ax.containers:
        ax.bar_label(container, fmt="  %d")

    plt.title("Amministrati in PA - IMPONIBILE (" + c + ")")
    plt.xlabel("")
    plt.ylabel("Totale imponibile / Numero cedolini")
    plt.tight_layout()

    fig = ax.get_figure()
    fig.savefig(dir_save + "IMPONIBILE_" + c + ".jpg")

## Cedolini

In [ ]:
dfCF21 = pd.read_csv(dir_data + "RitenuteFiscali.csv", index_col="Unnamed: 0")

dfCF21 = set_time(dfCF21)
dfCF21 = set_age(dfCF21, group=True)

dfCF22 = set_ral(dfCF21, ced=True)

dfCF23 = set_sex(dfCF22, col_ext=["ti", "eta"], drop=False)
dfCF23 = dfCF23.drop(columns="F/M")

dfCF23

In [ ]:
dfCG20[["F", "M"]].unstack(1).plot(ylim=(0, 2e6), subplots=False)
plt.legend(ncol=2)
plt.tight_layout()

## Contratti Gestiti

In [ ]:
dfCG20 = pd.read_csv(dir_data + "ContrattiGestiti.csv", index_col="Unnamed: 0")

dfCG20 = set_time(dfCG20)
dfCG20 = set_age(dfCG20, group=True)

dfCG20 = dfCG20.groupby(["ti", "sesso", "eta"]).sum().reset_index()

dfCG20 = set_sex(dfCG20, col_ext=["ti", "eta"], drop=False)
dfCG20 = dfCG20.drop(columns="F/M")

dfCG20

In [ ]:
dfCG20[["F", "M"]].unstack(1).plot(ylim=(0, 2e6), subplots=False)
plt.legend(ncol=2)
plt.tight_layout()

In [ ]:
dfCF21 = pd.read_csv(dir_data + "RitenuteFiscali.csv", index_col="Unnamed: 0")

dfCF21 = set_time(dfCF21)
dfCF21 = set_age(dfCF21, group=True)

dfCF22 = set_ral(dfCF21, tot=True)

dfCF23 = set_sex(dfCF22, col_ext=["ti", "eta"], drop=False)
dfCF23 = dfCF23.drop(columns="F/M")

dfCF23

In [ ]:
dfCF23[["F", "M"]].unstack(1).plot(ylim=(0, 4e10), subplots=False)
plt.legend(ncol=2)
plt.tight_layout()

In [ ]:
dfCF30 = dfCF23.merge(dfCG20, left_index=True, right_index=True)
dfCF30["M"] = dfCF30["M_x"] / dfCF30["M_y"]
dfCF30["F"] = dfCF30["F_x"] / dfCF30["F_y"]
dfCF30 = dfCF30.drop(columns=["M_x", "M_y", "F_x", "F_y"])
dfCF30

In [ ]:
dfCF31 = dfCF30.unstack(1)
dfCF31

In [ ]:
idx = pd.IndexSlice
for c in ["18-24", "25-44", "45-64", "65-100"]:
    dfCF31.loc[:, idx[["F", "M"], c]].plot(logy=False)
    plt.tight_layout()

# Misc

In [ ]:
ddf = pd.DataFrame({"A": [1, 2, 3, 4], "B": ["a", "b", "a", "b"], "C": ["q1", "w2", "e3", "r4"]})
ddf

In [ ]:
ddf.groupby(["B", "C"]).sum()

In [ ]:
ddf.pivot(index="B", columns="C", values="A")  # .sum()